In [1]:
# environment variables
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ["TRANSFORMERS_NO_TF"] = "1"



In [2]:
# %pip install tensorflow

import torch
import tensorflow as tf

# Both should return 'True'
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())
# Should recognize 1 GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

/opt/anaconda3/envs/hf-clean/lib/python3.9/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


True
True
Num GPUs Available:  1


In [3]:
# %pip install pandas scikit-learn
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

/opt/anaconda3/envs/hf-clean/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/hf-clean/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:


# #folder_path = r"C:\Users\raned\Documents\Github\POSTMODERATION\TrainingData"
# folder_path = r"TrainingData"
# print("Files in TrainingData folder:")
# print(os.listdir(folder_path))

# file_path = os.path.join(folder_path, "combined_balanced_dataset_BERT.csv")
# training_data_df = pd.read_csv(file_path)

Files in TrainingData folder:
['text.csv', 'training_data.csv', 'HateSpeechDatasetBalanced.csv', 'combined_balanced_dataset_BERT.csv', 'labeled_data.csv']


In [5]:

training_data_df = pd.read_csv("TrainingData/combined_training_data.csv")

In [6]:
pip list

Package                      Version
---------------------------- ------------------
absl-py                      2.3.0
accelerate                   0.23.0
appnope                      0.1.4
asttokens                    3.0.0
astunparse                   1.6.3
Bottleneck                   1.4.2
Brotli                       1.0.9
certifi                      2025.4.26
charset-normalizer           3.3.2
click                        8.1.8
comm                         0.2.2
debugpy                      1.8.11
decorator                    5.2.1
exceptiongroup               1.3.0
executing                    2.2.0
filelock                     3.17.0
flatbuffers                  25.2.10
fsspec                       2025.3.2
gast                         0.6.0
gmpy2                        2.2.1
google-pasta                 0.2.0
grpcio                       1.71.0
h5py                         3.13.0
hf-xet                       1.1.2
huggingface-hub              0.32.3
idna                     

In [6]:
pip list

Package                      Version
---------------------------- ------------------
absl-py                      2.3.0
accelerate                   0.23.0
appnope                      0.1.4
asttokens                    3.0.0
astunparse                   1.6.3
Brotli                       1.0.9
certifi                      2025.4.26
charset-normalizer           3.3.2
click                        8.1.8
comm                         0.2.2
debugpy                      1.8.11
decorator                    5.2.1
exceptiongroup               1.3.0
executing                    2.2.0
filelock                     3.17.0
flatbuffers                  25.2.10
fsspec                       2025.3.2
gast                         0.6.0
gmpy2                        2.2.1
google-pasta                 0.2.0
grpcio                       1.71.0
h5py                         3.13.0
huggingface_hub              0.29.2
idna                         3.10
importlib_metadata           8.7.0
ipykernel                 

In [7]:
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn


# training_data_df = pd.read_csv("/content/combined_balanced_dataset_BERT_abigailapp.csv")

# 3. Ensure 'Content' is cleaned and valid
training_data_df['Content'] = training_data_df['Content'].fillna('').astype(str)

# 4. Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    training_data_df['Content'].tolist(),
    training_data_df['Label'].tolist(),
    test_size=0.2,
    random_state=42
)


# 5. Optional: Use a subset of training data for faster prototyping
# train_texts = train_texts[:18000]
# train_labels = train_labels[:18000]


class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# 6. Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# 7. Tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# 8. PyTorch Dataset wrapper
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

# 9. Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=500,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    save_total_limit=2,
    logging_dir='./logs',
    fp16=torch.cuda.is_available(),  # Enable mixed precision if using GPU
)

# 🔁 10. Updated Metric Computation — per class
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision_not_malicious': precision[0],
        'recall_not_malicious': recall[0],
        'f1_not_malicious': f1[0],
        'precision_malicious': precision[1],
        'recall_malicious': recall[1],
        'f1_malicious': f1[1],
    }



# 2. Override the default loss
def compute_loss(model, inputs):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fct(logits, labels)
    return loss, outputs



class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss



trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    class_weights=class_weights
)



# 12. Train
trainer.train()

# 13. Evaluate
results = trainer.evaluate()
print("✅ Evaluation Results:", results)



/opt/anaconda3/envs/hf-clean/lib/python3.9/site-packages/huggingface_hub-0.29.2-py3.8.egg/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 33%|███▎      | 500/1506 [03:56<07:53,  2.13it/s]

{'loss': 0.237, 'learning_rate': 3.339973439575033e-05, 'epoch': 0.66}


 66%|██████▋   | 1000/1506 [07:53<03:59,  2.11it/s]

{'loss': 0.14, 'learning_rate': 1.6799468791500664e-05, 'epoch': 1.33}


                                                   
 66%|██████▋   | 1000/1506 [08:22<03:59,  2.11it/s]

...

{'loss': 0.0979, 'learning_rate': 1.99203187250996e-07, 'epoch': 1.99}


100%|██████████| 1506/1506 [12:24<00:00,  2.02it/s]


{'train_runtime': 744.3218, 'train_samples_per_second': 64.698, 'train_steps_per_second': 2.023, 'train_loss': 0.1578701177282954, 'epoch': 2.0}


100%|██████████| 189/189 [00:28<00:00,  6.57it/s]

✅ Evaluation Results: {'eval_loss': 0.17618580162525177, 'eval_accuracy': 0.9453488372093023, 'eval_precision_not_malicious': 0.9349865951742627, 'eval_recall_not_malicious': 0.9538461538461539, 'eval_f1_not_malicious': 0.9443222203418514, 'eval_precision_malicious': 0.9555335968379447, 'eval_recall_malicious': 0.9373182552504039, 'eval_f1_malicious': 0.9463382808677214, 'eval_runtime': 28.8931, 'eval_samples_per_second': 208.354, 'eval_steps_per_second': 6.541, 'epoch': 2.0}


In [ ]:
### Abigail's BERT code
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_data_df = pd.read_csv("/content/combined_balanced_dataset_BERT_abigailapp.csv")

# 3. Ensure 'Content' is cleaned and valid
training_data_df['Content'] = training_data_df['Content'].fillna('').astype(str)
training_data_df = shuffle(training_data_df, random_state=42)


# 4. Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    training_data_df['Content'].tolist(),
    training_data_df['Label'].tolist(),
    test_size=0.2,
    random_state=42
)

# 5. Optional: Use a subset of training data for faster prototyping
# train_texts = train_texts[:10000]
# train_labels = train_labels[:10000]

# 6. Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# 7. Tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# 8. PyTorch Dataset wrapper
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir='./results_2epoch',
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=250,
    save_steps=500,
    logging_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    save_total_limit=2,
    logging_dir='./logs',
    fp16=torch.cuda.is_available(),  # Enable mixed precision if using GPU
    # report_to="none"  # Disables WandB/etc. if not using them
    learning_rate=2e-5,  # much safer


)


# 🔁 10. Updated Metric Computation — per class
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision_not_malicious': precision[0],
        'recall_not_malicious': recall[0],
        'f1_not_malicious': f1[0],
        'precision_malicious': precision[1],
        'recall_malicious': recall[1],
        'f1_malicious': f1[1],
    }


# 11. Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)




# 12. Train
trainer.train()



# 13. Evaluate
results = trainer.evaluate()
print("✅ Evaluation Results:", results)






In [8]:
# 14. Save the trained model and tokenizer
model.save_pretrained("Bert_Model")
tokenizer.save_pretrained("Bert_Model")

print("✅ Model and tokenizer saved to 'Bert_Model/'")



✅ Model and tokenizer saved to 'Bert_Model/'


In [16]:
import os

print("Current working directory:", os.getcwd())
print("Saved files in 'Bert_Model':", os.listdir("Bert_Model"))



Current working directory: C:\Users\raned
Saved files in 'Bert_Model': ['config.json', 'model.safetensors', 'special_tokens_map.json', 'tokenizer_config.json', 'vocab.txt']


In [6]:

# 1. Define new texts
new_texts = [
    "Fuck this place.", 
    "What a beautiful day, feeling grateful!",
    "I hate it here.",
    "You're a piece of shit",
    "Dumb ass bitch",
    "you’re such a dumbass 🤡 nobody wants you around 💩", #testing malicious tweets with emojis
    "go back to your country 🖕",
    "You're such a clown 🤡 lol",
    "🖕🤡🖕"
]

# 2. Tokenize the new content
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# 3. Run the model in evaluation mode (no gradients)
model.eval()
with torch.no_grad():
    outputs = model(**new_encodings)
    predictions = torch.argmax(outputs.logits, dim=1)


# 4. Print results with readable labels
labels = ['Not Malicious', 'Malicious']
for text, pred in zip(new_texts, predictions):
    print(f"Text: {text}\nPrediction: {labels[pred]}\n")

Text: Fuck this place.
Prediction: Malicious

Text: What a beautiful day, feeling grateful!
Prediction: Not Malicious

Text: I hate it here.
Prediction: Not Malicious

Text: You're a piece of shit
Prediction: Malicious

Text: Dumb ass bitch
Prediction: Malicious

Text: you’re such a dumbass 🤡 nobody wants you around 💩
Prediction: Malicious

Text: go back to your country 🖕
Prediction: Not Malicious

Text: You're such a clown 🤡 lol
Prediction: Not Malicious

Text: 🖕🤡🖕
Prediction: Not Malicious

